Ưu tiên:
* check mỗi năm có bao nhiêu quốc gia có value (trừ bộ disaster)
* check mỗi quốc gia có đủ các indicator






In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

Theo thống kê trước đó của trang ClimateDashboard thì các bộ dữ liệu cùng có các năm từ 2013 - 2022

In [ ]:
year = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

## Các indicator

### Disaster

In [ ]:
an_disaster_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Physical_Risks_Climate_related_disasters_frequency.csv')

In [ ]:
an_disaster_df['Indicator'].unique()

array(['Climate related disasters frequency, Number of Disasters: Drought',
       'Climate related disasters frequency, Number of Disasters: Extreme temperature',
       'Climate related disasters frequency, Number of Disasters: Flood',
       'Climate related disasters frequency, Number of Disasters: Landslide',
       'Climate related disasters frequency, Number of Disasters: Storm',
       'Climate related disasters frequency, Number of Disasters: TOTAL',
       'Climate related disasters frequency, Number of Disasters: Wildfire'],
      dtype=object)

### Temperature

In [ ]:
an_temparature_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Annual Surface Temperature Change.csv')

In [ ]:
an_temparature_df['Indicator'].unique()

array(['Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980'],
      dtype=object)

### Sea level

In [ ]:
monthly_sea_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Change in Mean Sea Levels(raw).csv')

In [ ]:
monthly_sea_df['Indicator'].unique()

array(['Change in mean sea level: Sea level: TOPEX.Poseidon',
       'Change in mean sea level: Sea level: Jason.1',
       'Change in mean sea level: Sea level: Jason.2',
       'Change in mean sea level: Sea level: Jason.3',
       'Change in mean sea level: Sea level: Sentinel-6MF',
       'Change in mean sea level: Sea level: Trend'], dtype=object)

Biến đổi sang format từng năm:
1. Bỏ 'Change in mean sea level: Sea level: Trend'
2. Đổi các Indicator thành 1 Indicator 'Mean sea level'
3. Bắt đầu biến đổi format, giá trị của năm = mean giá trị của các tháng trong năm

### CO2

In [ ]:
monthly_CO2_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Atmospheric CO₂ Concentrations.csv')

In [ ]:
monthly_CO2_df['Indicator'].unique()

array(['Monthly Atmospheric Carbon Dioxide Concentrations',
       'Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change'],
      dtype=object)

Biển đổi sang format từng năm:
1. Đỗi với mỗi Indicator, bắt đầu biến đổi format, giá trị của năm = mean giá trị của các tháng trong năm


### Land Cover

In [ ]:
an_land_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Land_Cover_Accounts.csv')

In [ ]:
an_land_df['Indicator'].unique()

array(['Climate Altering Land Cover Index', 'Total Land Cover',
       'Artificial surfaces (including urban and associated areas)',
       'Grassland', 'Herbaceous crops', 'Inland water bodies',
       'Mangroves', 'Permanent snow and glaciers', 'Shrub-covered areas',
       'Shrubs and/or herbaceous vegetation, aquatic or regularly flooded',
       'Sparsely natural vegetated areas', 'Terrestrial barren land',
       'Tree-covered areas', 'Woody crops'], dtype=object)

### Forest and Carbon

In [ ]:
an_forest_carbon_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Forest_and_Carbon.csv')

In [ ]:
an_forest_carbon_df['Indicator'].unique()

array(['Carbon stocks in forests', 'Forest area',
       'Index of carbon stocks in forests', 'Index of forest extent',
       'Land area', 'Share of forest area'], dtype=object)

# Khám phá và Xử lý dữ liệu

In [ ]:
def mo_ta_co_ban (df):
  print(f'Số lượng dòng: {df.shape[0]}')
  print('=====================================')
  print(f'Số lượng cột: {df.shape[1]}')
  print('=====================================')
  print(f'{df.info()}')

In [ ]:
def thong_ke_co_ban(df, so_cot_bd_nam):
  unique_countries = df['Country'].nunique()
  unique_indicators = df['Indicator'].nunique()
  list_unique_indicators = df['Indicator'].unique()
  country_data_completeness = df.groupby('Country').count().loc[:, '2013':'2022']
  countries_with_complete_data = country_data_completeness.apply(lambda row: row.count() == 10, axis=1).sum()

  print(f'Có tổng cộng {unique_countries} quốc gia')
  print(f'Có tổng cộng {unique_indicators} chỉ số phụ')
  print(f'Cái chỉ số phụ: {list_unique_indicators}')

  print(f'\n=============================================================\n')

  print(f'Có tổng cộng {countries_with_complete_data} quốc gia có đủ năm từ F2013 đến F2022')

  print(f'--------------------')

  for year in df.columns[so_cot_bd_nam:]:
    non_zero = np.count_nonzero(df.groupby("Country")[year].count().values)
    print(year, "|Tổng số nước có:", non_zero)

  print(f'\n=============================================================\n')

In [ ]:
def kiem_tra_indicator(df, unique_indicators):
  # Biến đếm số lượng quốc gia có đủ tất cả các Indicator unique
  countries_with_all_indicators = 0

  # Duyệt qua tất cả các quốc gia trong dữ liệu
  for country in df['Country'].unique():
      # Lấy các chỉ số unique của quốc gia này
      country_indicators = df[df['Country'] == country]['Indicator'].unique()

      # Kiểm tra xem quốc gia này có đủ các chỉ số unique không
      missing_indicators = [indicator for indicator in unique_indicators if indicator not in country_indicators]

      if len(missing_indicators) == 0:
          print(f"Quốc gia {country} có đủ tất cả các Indicator unique.")
          countries_with_all_indicators += 1  # Tăng số lượng quốc gia đủ tất cả indicator
      else:
          print(f"Quốc gia {country} thiếu.")

  print(f'--------------------')
  # In ra số lượng quốc gia đủ tất cả các Indicator unique
  print(f"Tổng số quốc gia có đủ tất cả các Indicator unique: {countries_with_all_indicators}.")

In [ ]:
import pandas as pd

def find_indicators_with_min_countries(df, threshold):
    """
    Tìm các Indicator có ít nhất `threshold` quốc gia trong dữ liệu.

    Parameters:
    df (pandas.DataFrame): Dữ liệu chứa các cột 'Country' và 'Indicator'.
    threshold (int): Ngưỡng số quốc gia tối thiểu mà mỗi Indicator cần phải có.

    Returns:
    list: Danh sách các Indicator mà có ít nhất `threshold` quốc gia.
    """
    # Bước 1: Lấy danh sách tất cả các quốc gia unique
    all_countries = df['Country'].unique()
    num_countries = len(all_countries)

    # Bước 2: Lấy tất cả các giá trị unique của 'Indicator'
    unique_indicators = df['Indicator'].unique()

    # Bước 3: Kiểm tra xem mỗi indicator có đủ ít nhất `threshold` quốc gia không
    indicators_with_min_countries = []

    for indicator in unique_indicators:
        # Lọc các quốc gia có indicator này và không bị NaN
        countries_with_indicator = df[df['Indicator'] == indicator]['Country'].dropna().unique()

        # Nếu số lượng quốc gia có indicator này lớn hơn hoặc bằng threshold, thì thêm vào danh sách
        if len(countries_with_indicator) >= threshold:
            indicators_with_min_countries.append(indicator)

    return indicators_with_min_countries


## Annual Disaster Frequency

In [ ]:
an_disaster_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Physical_Risks_Climate_related_disasters_frequency.csv')

In [ ]:
mo_ta_co_ban(an_disaster_df)

Số lượng dòng: 975
Số lượng cột: 51
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 51 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ObjectId   975 non-null    int64  
 1   Country    975 non-null    object 
 2   ISO2       949 non-null    object 
 3   ISO3       975 non-null    object 
 4   Indicator  975 non-null    object 
 5   Unit       975 non-null    object 
 6   1980       140 non-null    float64
 7   1981       142 non-null    float64
 8   1982       126 non-null    float64
 9   1983       179 non-null    float64
 10  1984       133 non-null    float64
 11  1985       144 non-null    float64
 12  1986       132 non-null    float64
 13  1987       183 non-null    float64
 14  1988       183 non-null    float64
 15  1989       156 non-null    float64
 16  1990       199 non-null    float64
 17  1991       171 non-null    float64
 18  1992       157 non-null    float64
 19  1993       203

In [ ]:
an_disaster_df = an_disaster_df.replace(np.nan, 0)

In [ ]:
thong_ke_co_ban(an_disaster_df, 44)

Có tổng cộng 216 quốc gia
Có tổng cộng 7 chỉ số phụ
Cái chỉ số phụ: ['Climate related disasters frequency, Number of Disasters: Drought'
 'Climate related disasters frequency, Number of Disasters: Extreme temperature'
 'Climate related disasters frequency, Number of Disasters: Flood'
 'Climate related disasters frequency, Number of Disasters: Landslide'
 'Climate related disasters frequency, Number of Disasters: Storm'
 'Climate related disasters frequency, Number of Disasters: TOTAL'
 'Climate related disasters frequency, Number of Disasters: Wildfire']


Có tổng cộng 216 quốc gia có đủ năm từ F2013 đến F2022
--------------------
2018 |Tổng số nước có: 216
2019 |Tổng số nước có: 216
2020 |Tổng số nước có: 216
2021 |Tổng số nước có: 216
2022 |Tổng số nước có: 216
2023 |Tổng số nước có: 216
Source |Tổng số nước có: 216


Quốc gia Afghanistan, Islamic Rep. of có đủ tất cả các Indicator unique.
Quốc gia Albania có đủ tất cả các Indicator unique.
Quốc gia Algeria có đủ tất cả các Indicator

## Nhiệt độ bề mặt hằng năm

In [ ]:
an_temp_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Annual Surface Temperature Change.csv')

In [ ]:
# an_temp_df = an_temp_df.replace(np.nan, 0)

In [ ]:
mo_ta_co_ban(an_temp_df)

In [ ]:
thong_ke_co_ban(an_temp_df)

In [ ]:
# Bước 1: Kiểm tra số lượng quốc gia trong mỗi năm
for year in an_temp_df.columns[2:]:  # Bắt đầu từ cột thứ 3 (2013) đến 2022
    non_zero = np.count_nonzero(an_temp_df.groupby("Country")[year].count().values)
    print(f"Năm {year}: Tổng số quốc gia có dữ liệu là {non_zero}.")

# Bước 2: Kiểm tra mỗi quốc gia có đủ các Indicator unique không
# Lấy danh sách các giá trị unique của 'Indicator' trong toàn bộ dữ liệu
unique_indicators = an_temp_df['Indicator'].unique()

# Biến đếm số lượng quốc gia có đủ tất cả các Indicator unique
countries_with_all_indicators = 0

# Duyệt qua tất cả các quốc gia trong dữ liệu
for country in an_temp_df['Country'].unique():
    # Lấy các chỉ số unique của quốc gia này
    country_indicators = an_temp_df[an_temp_df['Country'] == country]['Indicator'].unique()

    # Kiểm tra xem quốc gia này có đủ các chỉ số unique không
    missing_indicators = [indicator for indicator in unique_indicators if indicator not in country_indicators]

    if len(missing_indicators) == 0:
        print(f"Quốc gia {country} có đủ tất cả các Indicator unique.")
        countries_with_all_indicators += 1  # Tăng số lượng quốc gia đủ tất cả indicator
    else:
        print(f"Quốc gia {country} thiếu các Indicator: {missing_indicators}.")

# In ra số lượng quốc gia đủ tất cả các Indicator unique
print(f"\nTổng số quốc gia có đủ tất cả các Indicator unique: {countries_with_all_indicators}.")


## Đất

In [ ]:
an_land_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Land_Cover_Accounts.csv')

In [ ]:
an_land_df['Indicator'].nunique()

14

In [ ]:
mo_ta_co_ban(an_land_df)

Số lượng dòng: 3033
Số lượng cột: 42
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033 entries, 0 to 3032
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             3033 non-null   int64  
 1   Country              3033 non-null   object 
 2   ISO2                 2964 non-null   object 
 3   ISO3                 3033 non-null   object 
 4   Indicator            3033 non-null   object 
 5   Unit                 3033 non-null   object 
 6   Source               3033 non-null   object 
 7   CTS_Code             3033 non-null   object 
 8   CTS_Name             3033 non-null   object 
 9   CTS_Full_Descriptor  3033 non-null   object 
 10  Climate_Influence    3033 non-null   object 
 11  F1992                3023 non-null   float64
 12  F1993                3027 non-null   float64
 13  F1994                3027 non-null   float64
 14  F1995                3027 non-null   float64
 15  F

In [ ]:
thong_ke_co_ban(an_land_df, 36)

TypeError: cannot do slice indexing on Index with these indexers [36] of type int

In [ ]:
list_indicator = find_indicators_with_min_countries(an_land_df, 200)

In [ ]:
list_indicator

['Climate Altering Land Cover Index',
 'Artificial surfaces (including urban and associated areas)',
 'Grassland',
 'Herbaceous crops',
 'Inland water bodies',
 'Mangroves',
 'Permanent snow and glaciers',
 'Shrub-covered areas',
 'Shrubs and/or herbaceous vegetation, aquatic or regularly flooded',
 'Sparsely natural vegetated areas',
 'Terrestrial barren land',
 'Tree-covered areas',
 'Woody crops']

In [ ]:
import pandas as pd

def find_optimal_indicators(df, num_countries=190):
    """
    Tìm các Indicator mà có mặt trong số `num_countries` quốc gia, đồng thời có số quốc gia tham gia nhiều nhất.

    Parameters:
    df (pandas.DataFrame): Dữ liệu chứa các cột 'Country' và 'Indicator'.
    num_countries (int): Số lượng quốc gia tham gia tối thiểu cho mỗi Indicator. Mặc định là 190 quốc gia.

    Returns:
    list: Danh sách các Indicator phù hợp.
    """
    # Bước 1: Lấy danh sách các quốc gia unique
    all_countries = df['Country'].unique()
    if len(all_countries) < num_countries:
        raise ValueError(f"Dữ liệu chỉ có {len(all_countries)} quốc gia, không đủ {num_countries} quốc gia.")

    # Bước 2: Tạo danh sách các Indicator unique
    unique_indicators = df['Indicator'].unique()

    # Bước 3: Duyệt qua từng Indicator để kiểm tra số quốc gia có dữ liệu
    indicators_with_country_count = []

    for indicator in unique_indicators:
        # Lọc ra các quốc gia có Indicator này (loại bỏ NaN)
        countries_with_indicator = df[df['Indicator'] == indicator]['Country'].dropna().unique()

        # Kiểm tra số quốc gia có Indicator này
        num_countries_with_indicator = len(countries_with_indicator)

        # Nếu có đủ quốc gia (>= num_countries) thì thêm vào danh sách
        if num_countries_with_indicator >= num_countries:
            indicators_with_country_count.append((indicator, num_countries_with_indicator))

    # Bước 4: Sắp xếp các Indicator theo số quốc gia tham gia (giảm dần)
    indicators_with_country_count.sort(key=lambda x: x[1], reverse=True)

    # Bước 5: Trả về danh sách các Indicator với số quốc gia tham gia nhiều nhất
    return indicators_with_country_count

# Ví dụ sử dụng hàm:
indicators_with_country_count = find_optimal_indicators(an_land_df, num_countries=190)

# In ra kết quả
if indicators_with_country_count:
    print("Các Indicator có mặt trong ít nhất 190 quốc gia:")
    for indicator, country_count in indicators_with_country_count:
        print(f"Indicator: {indicator}, Quốc gia tham gia: {country_count}")
else:
    print("Không có Indicator nào có đủ trong tất cả 190 quốc gia.")


Các Indicator có mặt trong ít nhất 190 quốc gia:
Indicator: Climate Altering Land Cover Index, Quốc gia tham gia: 243
Indicator: Artificial surfaces (including urban and associated areas), Quốc gia tham gia: 229
Indicator: Grassland, Quốc gia tham gia: 229
Indicator: Herbaceous crops, Quốc gia tham gia: 229
Indicator: Inland water bodies, Quốc gia tham gia: 229
Indicator: Mangroves, Quốc gia tham gia: 229
Indicator: Permanent snow and glaciers, Quốc gia tham gia: 229
Indicator: Shrub-covered areas, Quốc gia tham gia: 229
Indicator: Shrubs and/or herbaceous vegetation, aquatic or regularly flooded, Quốc gia tham gia: 229
Indicator: Sparsely natural vegetated areas, Quốc gia tham gia: 229
Indicator: Terrestrial barren land, Quốc gia tham gia: 229
Indicator: Tree-covered areas, Quốc gia tham gia: 229
Indicator: Woody crops, Quốc gia tham gia: 229


## Rừng và Carbon

In [ ]:
an_forest_carbon_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Forest_and_Carbon.csv')

In [ ]:
# Bước 1: Kiểm tra số lượng quốc gia trong mỗi năm
for year in an_forest_carbon_df.columns[2:]:  # Bắt đầu từ cột thứ 3 (2013) đến 2022
    non_zero = np.count_nonzero(an_forest_carbon_df.groupby("Country")[year].count().values)
    print(f"Năm {year}: Tổng số quốc gia có dữ liệu là {non_zero}.")

# Bước 2: Kiểm tra mỗi quốc gia có đủ các Indicator unique không
# Lấy danh sách các giá trị unique của 'Indicator' trong toàn bộ dữ liệu
unique_indicators = an_forest_carbon_df['Indicator'].unique()

# Biến đếm số lượng quốc gia có đủ tất cả các Indicator unique
countries_with_all_indicators = 0

# Duyệt qua tất cả các quốc gia trong dữ liệu
for country in an_forest_carbon_df['Country'].unique():
    # Lấy các chỉ số unique của quốc gia này
    country_indicators = an_forest_carbon_df[an_forest_carbon_df['Country'] == country]['Indicator'].unique()

    # Kiểm tra xem quốc gia này có đủ các chỉ số unique không
    missing_indicators = [indicator for indicator in unique_indicators if indicator not in country_indicators]

    if len(missing_indicators) == 0:
        print(f"Quốc gia {country} có đủ tất cả các Indicator unique.")
        countries_with_all_indicators += 1  # Tăng số lượng quốc gia đủ tất cả indicator
    else:
        print(f"Quốc gia {country} thiếu các Indicator: {missing_indicators}.")

# In ra số lượng quốc gia đủ tất cả các Indicator unique
print(f"\nTổng số quốc gia có đủ tất cả các Indicator unique: {countries_with_all_indicators}.")


## Mực nước biển

In [ ]:
monthly_sea_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Change in Mean Sea Levels(raw).csv')

In [ ]:
monthly_sea_df.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,Measure,Date,Value
0,1,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Poseidon,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate and Weather, Change in Mean Sea Level",Sea Okhotsk,D12/16/1992,10.11
1,2,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Poseidon,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate and Weather, Change in Mean Sea Level",Adriatic Sea,D12/17/1992,32.73
2,3,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Poseidon,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate and Weather, Change in Mean Sea Level",Andaman Sea,D12/17/1992,-14.88
3,4,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Poseidon,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate and Weather, Change in Mean Sea Level",Arabian Sea,D12/17/1992,-9.76
4,5,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Poseidon,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate and Weather, Change in Mean Sea Level",Atlantic Ocean,D12/17/1992,-13.66


In [ ]:
monthly_sea_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39617 entries, 0 to 39616
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             39617 non-null  int64  
 1   Country              39617 non-null  object 
 2   ISO2                 0 non-null      float64
 3   ISO3                 39617 non-null  object 
 4   Indicator            39617 non-null  object 
 5   Unit                 39617 non-null  object 
 6   Source               39617 non-null  object 
 7   CTS Code             39617 non-null  object 
 8   CTS Name             39617 non-null  object 
 9   CTS Full Descriptor  39617 non-null  object 
 10  Measure              39617 non-null  object 
 11  Date                 39617 non-null  object 
 12  Value                39617 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 3.9+ MB


In [ ]:
monthly_sea_df['Measure'].unique()

array(['Sea Okhotsk', 'Adriatic Sea', 'Andaman Sea', 'Arabian Sea',
       'Atlantic Ocean', 'Baltic Sea', 'Bay Bengal', 'Bering Sea',
       'Caribbean Sea', 'Gulf Mexico', 'Indian Ocean', 'Indonesian',
       'Mediterranean', 'Nino', 'North Pacific', 'North Sea',
       'Pacific Ocean', 'Sea Japan', 'South China', 'Southern Ocean',
       'Tropics', 'World', 'Yellow Sea', 'North Atlantic', 'Persian Gulf'],
      dtype=object)

**Đổi tên "South China" thành "South East Asia"**

In [ ]:
monthly_sea_df['Indicator'].unique()

array(['Change in mean sea level: Sea level: TOPEX.Poseidon',
       'Change in mean sea level: Sea level: Jason.1',
       'Change in mean sea level: Sea level: Jason.2',
       'Change in mean sea level: Sea level: Jason.3',
       'Change in mean sea level: Sea level: Sentinel-6MF',
       'Change in mean sea level: Sea level: Trend'], dtype=object)

In [ ]:
monthly_sea_df['Date'] = monthly_sea_df['Date'].astype(str)
# Xóa ký tự 'D' trong cột 'Date' và chuyển đổi sang định dạng datetime
monthly_sea_df['Date'] = monthly_sea_df['Date'].str.replace('D', '', regex=False)
monthly_sea_df['Date'] = pd.to_datetime(monthly_sea_df['Date'], format='%m/%d/%Y', errors='coerce')
monthly_sea_df['Year'] = monthly_sea_df['Date'].dt.year

# Pivot dữ liệu để chuyển các giá trị 'Year' thành cột và lấy giá trị trung bình theo 'Measure' và 'Year'
data_pivoted = monthly_sea_df.pivot_table(index=['Measure'], columns='Year', values='Value', aggfunc='mean')
data_pivoted = data_pivoted.reset_index()

# Lấy các cột bổ sung từ dữ liệu gốc và thêm vào dữ liệu đã pivot
additional_columns = monthly_sea_df[['Measure', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source']].drop_duplicates()
data_complete = pd.merge(additional_columns, data_pivoted, on='Measure', how='right')

# Đổi cột 'Indicator' thành một giá trị duy nhất và xóa các hàng trùng lặp
data_complete['Indicator'] = 'Change in mean sea level: Sea level'
data_unique = data_complete.drop_duplicates(subset=['Measure', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source'])

data_unique.head()

,Measure,Country,ISO2,ISO3,Indicator,Unit,Source,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Adriatic Sea,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-4.32,-43.352353,-14.461176,-18.405484,3.912353,7.342121,11.233030,7.281515,20.412353,22.538571,21.036338,1.876712,12.846806,-11.812923,1.332222,-6.528889,1.388889,24.776528,63.426081,26.725616,19.062754,41.813774,60.738108,41.870541,40.263000,5.071800,67.494595,48.928571,49.055676,43.623243,40.335890,67.538571,77.761429
6,Andaman Sea,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-12.13,-11.821176,-45.074118,1.073125,13.407879,-76.146667,-15.164848,13.432121,30.172941,26.985714,-10.415000,8.108333,4.563239,9.074923,-17.050556,-1.691622,66.049623,36.350685,67.815270,41.676712,66.257681,75.984727,46.396111,46.638378,118.080429,48.602549,59.548378,9.718649,86.420556,128.116486,120.405676,32.191429,20.615714
12,Arabian Sea,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,3.09,-1.883529,-1.110000,3.727500,-6.663030,11.391515,3.440000,-28.796364,-19.848235,-22.276667,-8.377143,7.372055,1.673889,15.496000,10.482222,19.335676,18.713208,36.622055,34.513108,37.030274,46.624203,59.758750,47.813889,76.339189,76.094143,66.067843,46.890811,81.558378,94.704865,92.790000,74.707568,88.901429,79.972857
18,Atlantic Ocean,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-12.31,-18.151176,-14.660000,-7.892258,-6.821765,-4.066061,0.809697,-2.917576,-1.501176,3.408571,4.932958,15.315068,14.209167,18.124308,19.177222,18.590811,22.581852,25.750946,27.064459,26.730000,34.438696,35.539821,37.596667,45.719189,49.254000,54.032353,60.154865,68.727838,67.125946,73.136081,73.060405,80.109286,75.495714
24,Baltic Sea,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,216.55,-36.814706,-12.332353,25.759375,-108.362121,-1.774242,34.204545,-7.334848,31.441176,-6.697222,-6.338571,-6.031918,41.441944,43.890308,-0.740000,121.669459,72.389245,-4.504384,-45.883919,71.070959,100.557101,-32.637321,-13.006667,143.338378,56.301714,119.694902,34.972973,80.689189,176.454865,71.869054,126.682329,122.545000,191.640000


In [ ]:
data_unique.to_csv('Anual_Change_Mean_Sea_Level.csv')

In [ ]:
import pandas as pd

# Định nghĩa các từ điển quốc gia giáp biển
pacific_ocean_asia = {'Philippines', 'Indonesia', 'Malaysia', 'Korea, Republic of', 'Japan', 'Taiwan', 'China',
                     'Singapore', "Korea, Democratic People's Republic of", 'Vietnam', 'Cambodia', 'Brunei Darussalam'}
sea_japan_asia = {'Korea, Republic of', 'Japan', "Korea, Democratic People's Republic of"}
southest_asia_sea = {'Taiwan', 'China', 'Vietnam', 'Cambodia', 'Philippines', 'Brunei Darussalam', 'Indonesia', 'Malaysia', 'Singapore'}
yellow_sea = {'Korea, Republic of', 'Japan', "Korea, Democratic People's Republic of"}

india_ocean_asia = {'India', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia', 'Yemen', 'Oman', "Iran, Islamic Republic of", 'Pakistan',
                    'Iraq', 'Kuwait', 'Saudi Arabia', 'Qatar', 'Bahrain', 'United Arab Emirates', 'Bangladesh', 'Sri Lanka', 'Maldives'}
andaman_sea_asia = {'India', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia'}
arabian_sea_asia = {'Yemen', 'Oman', "Iran, Islamic Republic of", 'Pakistan', 'India'}
persian_gulf_asia = {"Iran, Islamic Republic of", 'Pakistan', 'Iraq', 'Kuwait', 'Saudi Arabia', 'Qatar', 'Bahrain', 'United Arab Emirates', 'Oman'}
bay_bengal_asia = {'India', 'Bangladesh', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia', 'Sri Lanka'}

atlantic_ocean_asia = {'Georgia', 'Türkiye'}

mediterranean_asia = {'Türkiye', 'Cyprus', 'Syrian Arab Republic', 'Lebanon', 'Israel', "Palestine, State of"}

indonesian_asia = {'Indonesia', 'India', 'Myanmar', 'Thailand', 'Malaysia', 'Philippines', 'Taiwan', 'China',
                   'Vietnam', 'Cambodia', 'Brunei Darussalam', 'Singapore', 'Timor-Leste'}

north_pacific_asia = {'Japan', 'Korea, Republic of', 'Taiwan', 'China', "Korea, Democratic People's Republic of", 'Philippines'}

# Tạo một từ điển ánh xạ biển/đại dương với các quốc gia giáp biển
measure_to_countries = {
    "Pacific Ocean Asia": pacific_ocean_asia,
    "Sea of Japan Asia": sea_japan_asia,
    "Southeast Asia Sea": southest_asia_sea,
    "Yellow Sea": yellow_sea,
    "Indian Ocean Asia": india_ocean_asia,
    "Andaman Sea Asia": andaman_sea_asia,
    "Arabian Sea Asia": arabian_sea_asia,
    "Persian Gulf Asia": persian_gulf_asia,
    "Bay of Bengal Asia": bay_bengal_asia,
    "Atlantic Ocean Asia": atlantic_ocean_asia,
    "Mediterranean Asia": mediterranean_asia,
    "Indonesian Asia": indonesian_asia,
    "North Pacific Asia": north_pacific_asia
}

# Đọc dữ liệu của bạn vào pandas dataframe
data = pd.read_csv('/content/Anual_Change_Mean_Sea_Level.csv')  # Thay 'path_to_your_file.csv' bằng đường dẫn thực tế của bạn

# Hàm cập nhật cột 'Country'
def update_countries(row):
    measure = row['Measure']
    if measure in measure_to_countries:
        countries = measure_to_countries[measure]
        return ', '.join(countries)
    return row['Country']

# Áp dụng hàm để cập nhật cột 'Country'
data['Country'] = data.apply(update_countries, axis=1)

# Kiểm tra kết quả
print(data[['Measure', 'Country']].head())


          Measure Country
0    Adriatic Sea   World
1     Andaman Sea   World
2     Arabian Sea   World
3  Atlantic Ocean   World
4      Baltic Sea   World


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 41 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  25 non-null     int64  
 1   Measure     25 non-null     object 
 2   Country     25 non-null     object 
 3   ISO2        0 non-null      float64
 4   ISO3        25 non-null     object 
 5   Indicator   25 non-null     object 
 6   Unit        25 non-null     object 
 7   Source      25 non-null     object 
 8   1992        25 non-null     float64
 9   1993        25 non-null     float64
 10  1994        25 non-null     float64
 11  1995        25 non-null     float64
 12  1996        25 non-null     float64
 13  1997        25 non-null     float64
 14  1998        25 non-null     float64
 15  1999        25 non-null     float64
 16  2000        25 non-null     float64
 17  2001        25 non-null     float64
 18  2002        25 non-null     float64
 19  2003        25 non-null     flo

## CO2

In [ ]:
monthly_CO2_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Raw data/Atmospheric CO₂ Concentrations.csv')

In [ ]:
monthly_CO2_df.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,Date,Value
0,1,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M03,315.71
1,2,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M04,317.45
2,3,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M05,317.51
3,4,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M06,317.27
4,5,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M07,315.87


In [ ]:
monthly_CO2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             1582 non-null   int64  
 1   Country              1582 non-null   object 
 2   ISO2                 0 non-null      float64
 3   ISO3                 1582 non-null   object 
 4   Indicator            1582 non-null   object 
 5   Unit                 1582 non-null   object 
 6   Source               1582 non-null   object 
 7   CTS Code             1582 non-null   object 
 8   CTS Name             1582 non-null   object 
 9   CTS Full Descriptor  1582 non-null   object 
 10  Date                 1582 non-null   object 
 11  Value                1582 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 148.4+ KB


In [ ]:
monthly_CO2_df['Indicator'].unique()

array(['Monthly Atmospheric Carbon Dioxide Concentrations',
       'Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change'],
      dtype=object)

In [ ]:
monthly_CO2_df['Unit'].unique()

array(['Parts Per Million', 'Percent'], dtype=object)

In [ ]:
monthly_CO2_df_temp = monthly_CO2_df[monthly_CO2_df['Indicator'] == 'Monthly Atmospheric Carbon Dioxide Concentrations'].copy()
print(monthly_CO2_df_temp['Indicator'].unique())
print(monthly_CO2_df_temp['Unit'].unique())
monthly_CO2_df_temp.head()

['Monthly Atmospheric Carbon Dioxide Concentrations']
['Parts Per Million']


,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,Date,Value
0,1,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M03,315.71
1,2,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M04,317.45
2,3,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M05,317.51
3,4,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M06,317.27
4,5,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1958M07,315.87


In [ ]:
monthly_CO2_df_temp['Date'] = monthly_CO2_df_temp['Date'].astype(str)
# Xóa ký tự 'D' trong cột 'Date' và chuyển đổi sang định dạng datetime
monthly_CO2_df_temp['Date'] = monthly_CO2_df_temp['Date'].str.replace('M', '/', regex=False)
monthly_CO2_df_temp['Date'] = pd.to_datetime(monthly_CO2_df_temp['Date'], format='%Y/%m', errors='coerce')
monthly_CO2_df_temp['Year'] = monthly_CO2_df_temp['Date'].dt.year

# Pivot dữ liệu để chuyển các giá trị 'Year' thành cột và lấy giá trị trung bình theo 'Measure' và 'Year'
monthly_CO2_df_pivoted = monthly_CO2_df_temp.pivot_table(index=['Country'], columns='Year', values='Value', aggfunc='mean')
monthly_CO2_df_pivoted = monthly_CO2_df_pivoted.reset_index()

# Lấy các cột bổ sung từ dữ liệu gốc và thêm vào dữ liệu đã pivot
additional_columns = monthly_CO2_df_temp[['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source']].drop_duplicates()
monthly_CO2_df_complete = pd.merge(additional_columns, monthly_CO2_df_pivoted, on='Country', how='right')

# Đổi cột 'Indicator' thành một giá trị duy nhất và xóa các hàng trùng lặp
monthly_CO2_df_complete['Indicator'] = 'Monthly Atmospheric Carbon Dioxide Concentrations'
monthly_CO2_df_unique = monthly_CO2_df_complete.drop_duplicates(subset=['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source'])

monthly_CO2_df_unique.head()

,Country,ISO2,ISO3,Indicator,Unit,Source,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",315.237,315.981667,316.908333,317.643333,318.453333,318.9925,319.62,320.036667,321.3675,322.179167,323.05,324.620833,325.681667,326.32,327.456667,329.681667,330.193333,331.126667,332.025833,333.844167,335.415,336.835,338.7625,340.119167,341.478333,343.1525,344.868333,346.351667,347.6075,349.311667,351.69,353.205,354.4525,355.704167,356.545833,357.215,358.959167,360.968333,362.743333,363.876667,366.84,368.54,369.7075,371.319167,373.4525,375.983333,377.698333,379.983333,382.090833,384.025,385.8325,387.6425,390.101667,391.850833,394.055833,396.7375,398.8125,401.0125,404.4125,406.758333,408.715,411.649167,414.214167,416.414167,418.528333,421.075833,425.522857


In [ ]:
monthly_CO2_df_temp2 = monthly_CO2_df[monthly_CO2_df['Indicator'] == 'Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change'].copy()
print(monthly_CO2_df_temp2['Indicator'].unique())
print(monthly_CO2_df_temp2['Unit'].unique())
monthly_CO2_df_temp2.head()

['Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change']
['Percent']


,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,Date,Value
13,14,World,NaN,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1959M03,0.30
15,16,World,NaN,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1959M04,0.09
17,18,World,NaN,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1959M05,0.25
19,20,World,NaN,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1959M06,0.28
21,22,World,NaN,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",ECCA,Atmospheric Carbon Dioxide Concentrations,"Environment, Climate Change, Climate and Weather, Atmospheric Carbon Dioxide Concentrations",1959M07,0.21


In [ ]:
monthly_CO2_df_temp2['Date'] = monthly_CO2_df_temp2['Date'].astype(str)
# Xóa ký tự 'D' trong cột 'Date' và chuyển đổi sang định dạng datetime
monthly_CO2_df_temp2['Date'] = monthly_CO2_df_temp2['Date'].str.replace('M', '/', regex=False)
monthly_CO2_df_temp2['Date'] = pd.to_datetime(monthly_CO2_df_temp2['Date'], format='%Y/%m', errors='coerce')
monthly_CO2_df_temp2['Year'] = monthly_CO2_df_temp2['Date'].dt.year

# Pivot dữ liệu để chuyển các giá trị 'Year' thành cột và lấy giá trị trung bình theo 'Measure' và 'Year'
monthly_CO2_df_pivoted2 = monthly_CO2_df_temp2.pivot_table(index=['Country'], columns='Year', values='Value', aggfunc='mean')
monthly_CO2_df_pivoted2 = monthly_CO2_df_pivoted2.reset_index()

# Lấy các cột bổ sung từ dữ liệu gốc và thêm vào dữ liệu đã pivot
additional_columns = monthly_CO2_df_temp2[['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source']].drop_duplicates()
monthly_CO2_df_complete2 = pd.merge(additional_columns, monthly_CO2_df_pivoted2, on='Country', how='right')

# Đổi cột 'Indicator' thành một giá trị duy nhất và xóa các hàng trùng lặp
monthly_CO2_df_complete2['Indicator'] = 'Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change'
monthly_CO2_df_unique2 = monthly_CO2_df_complete2.drop_duplicates(subset=['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source'])

monthly_CO2_df_unique2.head()

,Country,ISO2,ISO3,Indicator,Unit,Source,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,World,NaN,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",0.234,0.2925,0.2325,0.255833,0.168333,0.195833,0.13,0.415,0.2525,0.270833,0.486667,0.326667,0.194167,0.348333,0.68,0.155,0.284167,0.270833,0.549167,0.47,0.424167,0.571667,0.400833,0.4,0.491667,0.4975,0.43,0.363333,0.490833,0.680833,0.43,0.354167,0.3525,0.2375,0.1875,0.488333,0.560833,0.4925,0.313333,0.815,0.463333,0.3175,0.435,0.575,0.68,0.454167,0.605833,0.555,0.505833,0.4725,0.468333,0.635,0.449167,0.563333,0.68,0.5225,0.5525,0.848333,0.58,0.4825,0.7175,0.623333,0.530833,0.508333,0.608333,0.847143


In [ ]:
an_CO2_concentrations = pd.concat([monthly_CO2_df_unique, monthly_CO2_df_unique2], ignore_index=True)

In [ ]:
an_CO2_concentrations.head()

,Country,ISO2,ISO3,Indicator,Unit,Source,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,World,NaN,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",315.237,315.981667,316.908333,317.643333,318.453333,318.992500,319.620000,320.036667,321.3675,322.179167,323.050000,324.620833,325.681667,326.320000,327.456667,329.681667,330.193333,331.126667,332.025833,333.844167,335.415,336.835000,338.762500,340.119167,341.478333,343.152500,344.868333,346.351667,347.607500,349.311667,351.690000,353.205,354.452500,355.704167,356.545833,357.2150,358.959167,360.968333,362.743333,363.876667,366.840,368.540000,369.7075,371.319167,373.4525,375.983333,377.698333,379.983333,382.090833,384.025000,385.8325,387.642500,390.101667,391.850833,394.055833,396.7375,398.8125,401.0125,404.412500,406.758333,408.7150,411.649167,414.214167,416.414167,418.528333,421.075833,425.522857
1,World,NaN,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,"Dr. Pieter Tans, National Oceanic and Atmospheric Administration (NOAA). Global Monitoring Laboratory, Trends in Atmospheric Carbon Dioxide Data (https://gml.noaa.gov/ccgg/trends/) and Dr. Ralph Keeling, Scripps Institution of Oceanography, Carbon Dioxide Measurements (https://scrippsco2.ucsd.edu/). Accessed on 2023-01-09.",NaN,0.234000,0.292500,0.232500,0.255833,0.168333,0.195833,0.130000,0.4150,0.252500,0.270833,0.486667,0.326667,0.194167,0.348333,0.680000,0.155000,0.284167,0.270833,0.549167,0.470,0.424167,0.571667,0.400833,0.400000,0.491667,0.497500,0.430000,0.363333,0.490833,0.680833,0.430,0.354167,0.352500,0.237500,0.1875,0.488333,0.560833,0.492500,0.313333,0.815,0.463333,0.3175,0.435000,0.5750,0.680000,0.454167,0.605833,0.555000,0.505833,0.4725,0.468333,0.635000,0.449167,0.563333,0.6800,0.5225,0.5525,0.848333,0.580000,0.4825,0.717500,0.623333,0.530833,0.508333,0.608333,0.847143


In [ ]:
an_CO2_concentrations.to_csv('Anual_Atmospheric_CO2_Concentrations.csv')

# Combine Dataset

Theo thống kê của IMF chỉ lấy từ năm 2013 - 2022 và lấy các quốc gia đều có trong các bộ dữ liệu

In [ ]:
def xu_ly_cot_nam(data: pd.DataFrame):
  data.columns = [
      col[1:] if col.startswith('F') and col[1:].isdigit() else col
      for col in data.columns
   ]
  return data

### Disaster

In [ ]:
an_disaster_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Processed Data/Physical_Risks_Climate_related_disasters_frequency.csv')

In [ ]:
an_disaster_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 51 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ObjectId   975 non-null    int64  
 1   Country    975 non-null    object 
 2   ISO2       949 non-null    object 
 3   ISO3       975 non-null    object 
 4   Indicator  975 non-null    object 
 5   Unit       975 non-null    object 
 6   1980       140 non-null    float64
 7   1981       142 non-null    float64
 8   1982       126 non-null    float64
 9   1983       179 non-null    float64
 10  1984       133 non-null    float64
 11  1985       144 non-null    float64
 12  1986       132 non-null    float64
 13  1987       183 non-null    float64
 14  1988       183 non-null    float64
 15  1989       156 non-null    float64
 16  1990       199 non-null    float64
 17  1991       171 non-null    float64
 18  1992       157 non-null    float64
 19  1993       203 non-null    float64
 20  1994      

In [ ]:
an_disaster_df = an_disaster_df[['Country', 'ISO3', 'Indicator', 'Unit'] + [f'{year}' for year in range(2013, 2023)]]

In [ ]:
an_disaster_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    975 non-null    object 
 1   ISO3       975 non-null    object 
 2   Indicator  975 non-null    object 
 3   Unit       975 non-null    object 
 4   2013       259 non-null    float64
 5   2014       241 non-null    float64
 6   2015       306 non-null    float64
 7   2016       257 non-null    float64
 8   2017       300 non-null    float64
 9   2018       291 non-null    float64
 10  2019       301 non-null    float64
 11  2020       311 non-null    float64
 12  2021       316 non-null    float64
 13  2022       299 non-null    float64
dtypes: float64(10), object(4)
memory usage: 106.8+ KB


### Temperature

In [ ]:
an_temparature_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Processed Data/Annual Surface Temperature Change.csv')

In [ ]:
an_temparature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 73 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             236 non-null    int64  
 1   Country              236 non-null    object 
 2   ISO2                 228 non-null    object 
 3   ISO3                 236 non-null    object 
 4   Indicator            236 non-null    object 
 5   Unit                 236 non-null    object 
 6   Source               236 non-null    object 
 7   CTS Code             236 non-null    object 
 8   CTS Name             236 non-null    object 
 9   CTS Full Descriptor  236 non-null    object 
 10  1961                 201 non-null    float64
 11  1962                 202 non-null    float64
 12  1963                 201 non-null    float64
 13  1964                 200 non-null    float64
 14  1965                 199 non-null    float64
 15  1966                 202 non-null    flo

In [ ]:
an_temparature_df = an_temparature_df[['Country', 'ISO3', 'Indicator', 'Unit'] + [f'{year}' for year in range(2013, 2023)]]

In [ ]:
an_temparature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    236 non-null    object 
 1   ISO3       236 non-null    object 
 2   Indicator  236 non-null    object 
 3   Unit       236 non-null    object 
 4   2013       221 non-null    float64
 5   2014       221 non-null    float64
 6   2015       223 non-null    float64
 7   2016       223 non-null    float64
 8   2017       223 non-null    float64
 9   2018       223 non-null    float64
 10  2019       222 non-null    float64
 11  2020       220 non-null    float64
 12  2021       223 non-null    float64
 13  2022       223 non-null    float64
dtypes: float64(10), object(4)
memory usage: 25.9+ KB


### Land Cover

In [ ]:
an_land_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Processed Data/Land_Cover_Accounts.csv')

In [ ]:
an_land_df = xu_ly_cot_nam(an_land_df)

In [ ]:
an_land_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033 entries, 0 to 3032
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             3033 non-null   int64  
 1   Country              3033 non-null   object 
 2   ISO2                 2964 non-null   object 
 3   ISO3                 3033 non-null   object 
 4   Indicator            3033 non-null   object 
 5   Unit                 3033 non-null   object 
 6   Source               3033 non-null   object 
 7   CTS_Code             3033 non-null   object 
 8   CTS_Name             3033 non-null   object 
 9   CTS_Full_Descriptor  3033 non-null   object 
 10  Climate_Influence    3033 non-null   object 
 11  1992                 3023 non-null   float64
 12  1993                 3027 non-null   float64
 13  1994                 3027 non-null   float64
 14  1995                 3027 non-null   float64
 15  1996                 3027 non-null   f

In [ ]:
an_land_df = an_land_df[['Country', 'ISO3', 'Indicator', 'Unit', 'Climate_Influence'] + [f'{year}' for year in range(2013, 2023)]]

In [ ]:
an_land_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033 entries, 0 to 3032
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country            3033 non-null   object 
 1   ISO3               3033 non-null   object 
 2   Indicator          3033 non-null   object 
 3   Unit               3033 non-null   object 
 4   Climate_Influence  3033 non-null   object 
 5   2013               3033 non-null   float64
 6   2014               3033 non-null   float64
 7   2015               3033 non-null   float64
 8   2016               3033 non-null   float64
 9   2017               3033 non-null   float64
 10  2018               3030 non-null   float64
 11  2019               3033 non-null   float64
 12  2020               3033 non-null   float64
 13  2021               3033 non-null   float64
 14  2022               3033 non-null   float64
dtypes: float64(10), object(5)
memory usage: 355.6+ KB


### Forest and Carbon

In [ ]:
an_forest_carbon_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Processed Data/Forest_and_Carbon.csv')

In [ ]:
an_forest_carbon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333 entries, 0 to 1332
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             1333 non-null   int64  
 1   Country              1333 non-null   object 
 2   ISO2                 1287 non-null   object 
 3   ISO3                 1333 non-null   object 
 4   Indicator            1333 non-null   object 
 5   Unit                 1333 non-null   object 
 6   Source               1333 non-null   object 
 7   CTS_Code             1333 non-null   object 
 8   CTS_Name             0 non-null      float64
 9   CTS_Full_Descriptor  1333 non-null   object 
 10  F1992                1313 non-null   float64
 11  F1993                1321 non-null   float64
 12  F1994                1321 non-null   float64
 13  F1995                1321 non-null   float64
 14  F1996                1321 non-null   float64
 15  F1997                1321 non-null   f

In [ ]:
an_forest_carbon_df = xu_ly_cot_nam(an_forest_carbon_df)

In [ ]:
an_forest_carbon_df = an_forest_carbon_df[['Country', 'ISO3', 'Indicator', 'Unit'] + [f'{year}' for year in range(2013, 2023)]]

In [ ]:
an_forest_carbon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333 entries, 0 to 1332
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    1333 non-null   object 
 1   ISO3       1333 non-null   object 
 2   Indicator  1333 non-null   object 
 3   Unit       1333 non-null   object 
 4   2013       1321 non-null   float64
 5   2014       1321 non-null   float64
 6   2015       1321 non-null   float64
 7   2016       1321 non-null   float64
 8   2017       1321 non-null   float64
 9   2018       1321 non-null   float64
 10  2019       1321 non-null   float64
 11  2020       1321 non-null   float64
 12  2021       911 non-null    float64
 13  2022       911 non-null    float64
dtypes: float64(10), object(4)
memory usage: 145.9+ KB


### Sea level

In [ ]:
an_sea_level_df = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Processed Data/Anual_Change_Mean_Sea_Level.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Processed Data/Anual_Change_Mean_Sea_Level.csv'

In [ ]:
an_sea_level_df.info()

NameError: name 'an_sea_level_df' is not defined

In [ ]:
an_sea_level_df.head()

In [ ]:
an_sea_level_df = an_sea_level_df[['Country', 'Measure', 'ISO3', 'Indicator', 'Unit'] + [f'{year}' for year in range(2013, 2023)]]

In [ ]:
an_sea_level_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    25 non-null     object 
 1   Measure    25 non-null     object 
 2   ISO3       25 non-null     object 
 3   Indicator  25 non-null     object 
 4   Unit       25 non-null     object 
 5   2013       25 non-null     float64
 6   2014       25 non-null     float64
 7   2015       25 non-null     float64
 8   2016       25 non-null     float64
 9   2017       25 non-null     float64
 10  2018       25 non-null     float64
 11  2019       25 non-null     float64
 12  2020       25 non-null     float64
 13  2021       25 non-null     float64
 14  2022       25 non-null     float64
dtypes: float64(10), object(5)
memory usage: 3.1+ KB


In [ ]:
an_sea_level_df.head()

,Country,Measure,ISO3,Indicator,Unit,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,World,Adriatic Sea,WLD,Change in mean sea level: Sea level,Millimeters,41.813774,60.738108,41.870541,40.263000,5.071800,67.494595,48.928571,49.055676,43.623243,40.335890
1,World,Andaman Sea,WLD,Change in mean sea level: Sea level,Millimeters,75.984727,46.396111,46.638378,118.080429,48.602549,59.548378,9.718649,86.420556,128.116486,120.405676
2,World,Arabian Sea,WLD,Change in mean sea level: Sea level,Millimeters,59.758750,47.813889,76.339189,76.094143,66.067843,46.890811,81.558378,94.704865,92.790000,74.707568
3,World,Atlantic Ocean,WLD,Change in mean sea level: Sea level,Millimeters,35.539821,37.596667,45.719189,49.254000,54.032353,60.154865,68.727838,67.125946,73.136081,73.060405
4,World,Baltic Sea,WLD,Change in mean sea level: Sea level,Millimeters,-32.637321,-13.006667,143.338378,56.301714,119.694902,34.972973,80.689189,176.454865,71.869054,126.682329


### CO2

In [ ]:
an_CO2_concentrations = pd.read_csv('/content/drive/MyDrive/[Đạt]_DS105/Đồ Án DS105/[1]_Data_[28_10]/Processed Data/Anual_Atmospheric_CO2_Concentrations.csv')

In [ ]:
an_CO2_concentrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 74 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2 non-null      int64  
 1   Country     2 non-null      object 
 2   ISO2        0 non-null      float64
 3   ISO3        2 non-null      object 
 4   Indicator   2 non-null      object 
 5   Unit        2 non-null      object 
 6   Source      2 non-null      object 
 7   1958        1 non-null      float64
 8   1959        2 non-null      float64
 9   1960        2 non-null      float64
 10  1961        2 non-null      float64
 11  1962        2 non-null      float64
 12  1963        2 non-null      float64
 13  1964        2 non-null      float64
 14  1965        2 non-null      float64
 15  1966        2 non-null      float64
 16  1967        2 non-null      float64
 17  1968        2 non-null      float64
 18  1969        2 non-null      float64
 19  1970        2 non-null      float

In [ ]:
an_CO2_concentrations = an_CO2_concentrations[['Country', 'ISO3', 'Indicator', 'Unit'] + [f'{year}' for year in range(2013, 2023)]]

In [ ]:
an_CO2_concentrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    2 non-null      object 
 1   ISO3       2 non-null      object 
 2   Indicator  2 non-null      object 
 3   Unit       2 non-null      object 
 4   2013       2 non-null      float64
 5   2014       2 non-null      float64
 6   2015       2 non-null      float64
 7   2016       2 non-null      float64
 8   2017       2 non-null      float64
 9   2018       2 non-null      float64
 10  2019       2 non-null      float64
 11  2020       2 non-null      float64
 12  2021       2 non-null      float64
 13  2022       2 non-null      float64
dtypes: float64(10), object(4)
memory usage: 352.0+ bytes


In [ ]:
an_CO2_concentrations.head()

,Country,ISO3,Indicator,Unit,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,World,WLD,Monthly Atmospheric Carbon Dioxide Concentrations,Parts Per Million,396.7375,398.8125,401.0125,404.412500,406.758333,408.7150,411.649167,414.214167,416.414167,418.528333
1,World,WLD,"Monthly Atmospheric Carbon Dioxide Concentrations, Year on Year Percentage Change",Percent,0.6800,0.5225,0.5525,0.848333,0.580000,0.4825,0.717500,0.623333,0.530833,0.508333


### combine

In [ ]:
# Nối các DataFrame lại với nhau
df_merged = pd.concat([an_disaster_df, an_CO2_concentrations, an_forest_carbon_df, an_land_df,
                       an_sea_level_df, an_temparature_df], axis=0, ignore_index=True)

# Đặt thứ tự cột mong muốn
desired_order = ['Country', 'ISO3', 'Indicator', 'Measure', 'Climate_Influence', 'Unit'] + [f'{year}' for year in range(2013, 2023)]
df_merged = df_merged.reindex(columns=desired_order)

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5604 entries, 0 to 5603
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country            5604 non-null   object 
 1   ISO3               5604 non-null   object 
 2   Indicator          5604 non-null   object 
 3   Measure            25 non-null     object 
 4   Climate Influence  0 non-null      float64
 5   Unit               5604 non-null   object 
 6   2013               4861 non-null   float64
 7   2014               4843 non-null   float64
 8   2015               4910 non-null   float64
 9   2016               4861 non-null   float64
 10  2017               4904 non-null   float64
 11  2018               4892 non-null   float64
 12  2019               4904 non-null   float64
 13  2020               4912 non-null   float64
 14  2021               4510 non-null   float64
 15  2022               4493 non-null   float64
dtypes: float64(11), object(5

In [ ]:
df_merged.head()

,Country,ISO3,Indicator,Measure,Climate Influence,Unit,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of Disasters: Drought",NaN,NaN,Number of,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of Disasters: Extreme temperature",NaN,NaN,Number of,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of Disasters: Flood",NaN,NaN,Number of,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of Disasters: Landslide",NaN,NaN,Number of,1.0,NaN,4.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0
4,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of Disasters: Storm",NaN,NaN,Number of,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN


In [ ]:
df_merged.to_csv('combined_data.csv', index=False)

In [ ]:
{'pacific_ocean_asia': ['Philippines', 'Indonesia', 'Malaysia', 'Korea, Republic of', 'Japan', 'Taiwan', 'China',
                     'Singapore', "Korea, Democratic People's Republic of", 'Vietnam', 'Cambodia', 'Brunei Darussalam'],
'sea_japan_asia' : ['Korea, Republic of', 'Japan', "Korea, Democratic People's Republic of"],
'southest_asia_sea' : ['Taiwan', 'China', 'Vietnam', 'Cambodia', 'Philippines', 'Brunei Darussalam', 'Indonesia', 'Malaysia', 'Singapore'],
'yellow_sea' : ['Korea, Republic of', 'Japan', "Korea, Democratic People's Republic of"],

'india_ocean_asia' : ['India', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia', 'Yemen', 'Oman', "Iran, Islamic Republic of", 'Pakistan',
                    'Iraq', 'Kuwait', 'Saudi Arabia', 'Qatar', 'Bahrain', 'United Arab Emirates', 'Bangladesh', 'Sri Lanka', 'Maldives'],
'andaman_sea_asia' : ['India', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia'],
'arabian_sea_asia' : ['Yemen', 'Oman', "Iran, Islamic Republic of", 'Pakistan', 'India'],
'persian_gulf_asia' : ["Iran, Islamic Republic of", 'Pakistan', 'Iraq', 'Kuwait', 'Saudi Arabia', 'Qatar', 'Bahrain', 'United Arab Emirates', 'Oman'],
'bay_bengal_asia' : ['India', 'Bangladesh', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia', 'Sri Lanka'],

'atlantic_ocean_asia' : ['Georgia', 'Türkiye'],

'mediterranean_asia' : ['Türkiye', 'Cyprus', 'Syrian Arab Republic', 'Lebanon', 'Israel', "Palestine, State of"],

'indonesian_asia' : ['Indonesia', 'India', 'Myanmar', 'Thailand', 'Malaysia', 'Philippines', 'Taiwan', 'China',
                   'Vietnam', 'Cambodia', 'Brunei Darussalam', 'Singapore', 'Timor-Leste'],

'north_pacific_asia' : ['Japan', 'Korea, Republic of', 'Taiwan', 'China', "Korea, Democratic People's Republic of", 'Philippines']}

In [ ]:
df = pd.read_csv('/content/Anual_Change_Mean_Sea_Level.csv')

In [ ]:
df['Measure'].unique()

array(['Adriatic Sea', 'Andaman Sea', 'Arabian Sea', 'Atlantic Ocean',
       'Baltic Sea', 'Bay Bengal', 'Bering Sea', 'Caribbean Sea',
       'Gulf Mexico', 'Indian Ocean', 'Indonesian', 'Mediterranean',
       'Nino', 'North Atlantic', 'North Pacific', 'North Sea',
       'Pacific Ocean', 'Persian Gulf', 'Sea Japan', 'Sea Okhotsk',
       'South China', 'Southern Ocean', 'Tropics', 'World', 'Yellow Sea'],
      dtype=object)

In [ ]:
import pandas as pd



data = {
    'Pacific Ocean': ['Philippines', 'Indonesia', 'Malaysia', 'Korea, Republic of', 'Japan', 'Taiwan', 'China',
                           'Singapore', "Korea, Democratic People's Republic of", 'Vietnam', 'Cambodia', 'Brunei Darussalam'],
    'Sea Japan': ['Korea, Republic of', 'Japan', "Korea, Democratic People's Republic of"],
    'South China': ['Taiwan', 'China', 'Vietnam', 'Cambodia', 'Philippines', 'Brunei Darussalam', 'Indonesia', 'Malaysia', 'Singapore'],
    'Yellow Sea': ['Korea, Republic of', 'Japan', "Korea, Democratic People's Republic of"],
    'Indian Ocean': ['India', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia', 'Yemen', 'Oman', "Iran, Islamic Republic of", 'Pakistan',
                         'Iraq', 'Kuwait', 'Saudi Arabia', 'Qatar', 'Bahrain', 'United Arab Emirates', 'Bangladesh', 'Sri Lanka', 'Maldives'],
    'Andaman Sea': ['India', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia'],
    'Arabian Sea': ['Yemen', 'Oman', "Iran, Islamic Republic of", 'Pakistan', 'India'],
    'Persian Gulf': ["Iran, Islamic Republic of", 'Pakistan', 'Iraq', 'Kuwait', 'Saudi Arabia', 'Qatar', 'Bahrain', 'United Arab Emirates', 'Oman'],
    'Bay Bengal': ['India', 'Bangladesh', 'Myanmar', 'Thailand', 'Indonesia', 'Malaysia', 'Sri Lanka'],
    'Atlantic Ocean': ['Georgia', 'Türkiye'],
    'Mediterranean': ['Türkiye', 'Cyprus', 'Syrian Arab Republic', 'Lebanon', 'Israel', "Palestine, State of"],
    'Indonesian': ['Indonesia', 'India', 'Myanmar', 'Thailand', 'Malaysia', 'Philippines', 'Taiwan', 'China',
                        'Vietnam', 'Cambodia', 'Brunei Darussalam', 'Singapore', 'Timor-Leste'],
    'North Pacific': ['Japan', 'Korea, Republic of', 'Taiwan', 'China', "Korea, Democratic People's Republic of", 'Philippines']
}

# Hàm tạo thêm các dòng mẫu với quốc gia tương ứng
def create_sample_rows(df, data):
    new_rows = []  # Danh sách chứa các dòng mẫu mới
    for sea, countries in data.items():
        for country in countries:
            # Sao chép các thông tin cột khác và thêm quốc gia mới cho mỗi đại dương
            new_row = df[df['Measure'] == sea].copy()  # Lọc các dòng có cùng 'sea'
            new_row['country'] = country  # Thêm cột 'country' với giá trị quốc gia
            new_rows.append(new_row)

    # Nối các dòng mới vào DataFrame gốc
    new_df = pd.concat(new_rows, ignore_index=True)
    return new_df



# Cập nhật DataFrame với quốc gia mới
df_updated = create_sample_rows(df, data)

# In kết quả
df_updated


,Unnamed: 0,Measure,Country,ISO2,ISO3,Indicator,Unit,Source,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,country
0,96,Pacific Ocean,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-22.20,-18.485294,-12.876471,-10.025806,-5.076471,2.475758,-7.115152,-6.166667,0.985294,7.261111,11.361429,10.043836,14.640278,17.389231,19.136111,17.648649,19.838889,23.889041,19.700000,18.923288,32.366667,32.264286,41.052778,51.037838,55.170000,53.094118,58.545946,63.616216,64.008378,68.667162,68.403649,77.200714,63.471429,Philippines
1,96,Pacific Ocean,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-22.20,-18.485294,-12.876471,-10.025806,-5.076471,2.475758,-7.115152,-6.166667,0.985294,7.261111,11.361429,10.043836,14.640278,17.389231,19.136111,17.648649,19.838889,23.889041,19.700000,18.923288,32.366667,32.264286,41.052778,51.037838,55.170000,53.094118,58.545946,63.616216,64.008378,68.667162,68.403649,77.200714,63.471429,Indonesia
2,96,Pacific Ocean,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-22.20,-18.485294,-12.876471,-10.025806,-5.076471,2.475758,-7.115152,-6.166667,0.985294,7.261111,11.361429,10.043836,14.640278,17.389231,19.136111,17.648649,19.838889,23.889041,19.700000,18.923288,32.366667,32.264286,41.052778,51.037838,55.170000,53.094118,58.545946,63.616216,64.008378,68.667162,68.403649,77.200714,63.471429,Malaysia
3,96,Pacific Ocean,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-22.20,-18.485294,-12.876471,-10.025806,-5.076471,2.475758,-7.115152,-6.166667,0.985294,7.261111,11.361429,10.043836,14.640278,17.389231,19.136111,17.648649,19.838889,23.889041,19.700000,18.923288,32.366667,32.264286,41.052778,51.037838,55.170000,53.094118,58.545946,63.616216,64.008378,68.667162,68.403649,77.200714,63.471429,"Korea, Republic of"
4,96,Pacific Ocean,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-22.20,-18.485294,-12.876471,-10.025806,-5.076471,2.475758,-7.115152,-6.166667,0.985294,7.261111,11.361429,10.043836,14.640278,17.389231,19.136111,17.648649,19.838889,23.889041,19.700000,18.923288,32.366667,32.264286,41.052778,51.037838,55.170000,53.094118,58.545946,63.616216,64.008378,68.667162,68.403649,77.200714,63.471429,Japan
5,96,Pacific Ocean,World,NaN,WLD,Change in mean sea level: Sea level,Millimeters,National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08.,-22.20,-18.485294,-12.876471,-10.025806,-5.076471,2.475758,-7.115152,-6.166667,0.985294,7.261111,11.361429,10.043836,14.640278,17.389231,19.136111,17.648649,19.838889,23.889041,19.700000,18.923288,32.366667,32.264286,41.052778,51.037838,55.170000,53.094118,58.545946,63.616216,64.008378